In [87]:
import jieba
import numpy as np
import pandas as pd
import math
import re
import csv
import networkx
import matplotlib.pyplot as plt

In [15]:
articles = pd.read_csv('gcore_article.csv',encoding='gb18030')
print(articles.shape)

(1800, 5)


In [8]:
with open('stopwords.txt','r') as f:
    stop_words = f.readlines()
for i in range(len(stop_words)):
    stop_words[i] = stop_words[i].rstrip('\n')

In [16]:
for i in range(articles.shape[0]):
    if pd.isna(articles['content'][i]):
        articles['content'][i] = ''
    cut_list = [word for word in jieba.cut_for_search(articles['content'][i]) if word not in stop_words]
    articles['content'][i] = cut_list

In [17]:
for i in range(articles.shape[0]):
    cut_list = [word for word in jieba.cut_for_search(articles['title'][i]) if word not in stop_words]
    articles['title'][i] = cut_list

In [60]:
for i in range(articles.shape[0]):
    t = articles['links'][i]
    t = t.strip('[').strip(']').split(',')
    for j in range(len(t)):
        t[j] = t[j].strip(' ').lstrip('\'').rstrip('\'')
        t[j] = t[j].replace('https://www.gcores.com/link?target=','')
    t = [link for link in t if "https://www.gcores.com/articles/" in link]
    articles['links'][i] = t

In [91]:
url_dict = {}
url_list = []
pagegraph = networkx.DiGraph()
for i in range(articles.shape[0]):
    url_list.append(articles['url'][i])
    url_dict[articles['url'][i]] = articles['links'][i]
for url, links in url_dict.items():
    for link in links:
        if link in url_list:
            pagegraph.add_edge(url, link)
pr = networkx.pagerank(pagegraph, alpha=0.85)

In [92]:
page_rank_df = pd.Series(pr, name='page_rank')
page_rank_df = page_rank_df.apply(lambda x: math.log(x * 10000, 10) + 1)
page_rank_df.index.name = 'url'
page_rank_df.to_csv('page_rank.csv')

In [93]:
csv_headers = ['title','url','date','content']
with open('gcore_article_cut.csv', 'w', encoding='gb18030', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(csv_headers)

In [94]:
for i in range(articles.shape[0]):
    s1 = ' '.join(articles['title'][i])
    s2 = ' '.join(articles['content'][i])
    article_info = {
        "title": s1,
        "url": articles['url'][i],
        "date": articles['date'][i],
        "content": s2,
    }
    with open('gcore_article_cut.csv', 'a', encoding='gb18030', newline='') as f:
        csv_writer = csv.DictWriter(f, fieldnames=article_info.keys())
        csv_writer.writerow(article_info)